In [1]:
# Install packages from requirements.txt

# %pip install -r requirements.txt

# imports

In [3]:
# Set the working directory and adjust the Python path
import os
import sys

# Set the working directory to the src directory
os.chdir('./')
sys.path.append(os.getcwd())

from scripts.utils.generators import *
from scripts.utils.toy_city_generators import *
from configuration.algorithm_settings import settings

from collections import Counter
from time import time
from scripts.utils.generators import generate_pheromone_map
import numpy as np
from scripts.utils.roulette_selection import roulette_wheel_selection

import pprint


# get graph map

In [4]:
size=10
fixed_weight=0.1

map_graph = generate_square_city_graph(size,fixed_weight)

In [5]:
buses_graph = generate_bus_line_square_city(size,fixed_weight)

In [6]:
map_graph = merge_bus_and_map_graph(map_graph,buses_graph)

# run ACO algorithms

## Ant MAX MIN

In [28]:
def ant_solution_MAXMIN(graph_map: dict, pheromone_graph: dict, start_node: int, end_node: int, transition_prob: float, heuristic_weight: float, pheromone_weight: float):
    """
    Finds a solution path for an ant using the MAX-MIN Ant System.

    Parameters:
    adj_matrix: Adjacency matrix where each arc (row i, column j) has the cost
    pheromone_matrix: Pheromone matrix for each edge
    start_node: Root node (ant nest)
    end_node: Destination node (food)
    transition_prob :  Random state transition parameter between [0,1]
    alpha and beta: Weigh the importance of the heuristic and pheromone values

    Returns:
    path: Solution (path) found by the ant, including total cost at the end
    """

    solution_path = [start_node]
    solution_cost = 0

    while solution_path[-1] != end_node:
        current_node = solution_path[-1]
        neighbors = np.array(graph_map["connections"][current_node])
        neighbors_weights = np.array(graph_map["weights"][current_node])
        neighbors_pheromones = np.array(pheromone_graph[current_node])

        filter_visited_nodes_mask = ~np.isin(neighbors, solution_path)
        neighbors = neighbors[filter_visited_nodes_mask]
        neighbors_weights = neighbors_weights[filter_visited_nodes_mask]
        neighbors_pheromones = neighbors_pheromones[filter_visited_nodes_mask]

         # The ant is lost. Stop the search
        if len(neighbors) == 0:
            solution_path.append(np.inf)
            break

        # Probabilistic choice of the next node (proposed by Ant Colony System ACS)
        q = np.random.rand()
        if q <= transition_prob:
            Z = (neighbors_pheromones ** heuristic_weight) * ((1.0 / neighbors_weights) ** pheromone_weight)
            next_node = neighbors[np.argmax(Z)]
            solution_path.append(next_node)
        else:
            # total_attractiveness = np.sum((pheromone_matrix[current_node, neighbors] ** alpha) * ((1.0 / adj_matrix[current_node, neighbors]) ** beta))
            pheromone_values = neighbors_pheromones ** heuristic_weight
            heuristic_values = (1.0 / neighbors_weights) ** pheromone_weight
            total_attractiveness = np.sum(pheromone_values * heuristic_values)
            probabilities = (pheromone_values * heuristic_values) / total_attractiveness

            # Select the next node based on the roulette wheel selection
            next_node_index = roulette_wheel_selection(probabilities)
            solution_path.append(neighbors[next_node_index-1])

    # return the path and calculate the incurred costs
    if solution_path[-1] != np.inf:
        for i in range(len(solution_path) - 1):
            neighbor_selected_index = graph_map["connections"][solution_path[i]].index(solution_path[i + 1])
            solution_cost += graph_map["weights"][solution_path[i]][neighbor_selected_index]
    else:
        solution_cost = np.inf

    return solution_path, solution_cost


In [29]:
def ACS_MAXMIN(graph_map: dict, start_node: int, end_node: int, ants_number: int, global_evap_rate:float, transition_prob:float, max_epochs, initial_pheromone_lvl: float, heuristic_weight:float, pheromone_weight:float):
    """
    Ant Colony System with MAX-MIN strategy.

    Parameters:
    adj_matrix: Adjacency matrix where each arc (row i, column j) has the cost
    start_node: Root node (ant nest)
    end_node: Destination node (food)
    num_ants: Number of ants for the experiment
    p: Pheromone evaporation rate between [0,1]
    q0: Random state transition parameter between [0,1]
    max_epochs: Maximum number of epochs to run the algorithm
    initial_pheromone: Initial pheromone level on all edges
    alpha and beta: Parameters to weigh the importance of heuristic and pheromone values

    Returns:
    path: Optimal path found by the algorithm
    costo_MejorGlobal: Total distance of the optimal path
    t: Time taken by the algorithm to complete
    epocas: Number of epochs executed
    """

    pheromone_graph= generate_pheromone_map(graph_map,initial_pheromone_lvl)
    routes = [None] * ants_number  # Paths taken by each ant
    distances = np.zeros(ants_number)

    epochs = 0
    counter = 0

    tic = time.time()
    while counter < ants_number:
        # Each ant performs its tour
        for ant in range(ants_number):
            path_found, path_distance  = ant_solution_MAXMIN(graph_map, pheromone_graph, start_node, end_node, transition_prob, heuristic_weight, pheromone_weight)
            routes[ant] = path_found
            distances[ant] = path_distance

        # Global pheromone evaporation
        for key in pheromone_graph:
            pheromone_graph[key] *= (1 - global_evap_rate)

        # Sort ants based on path distances
        sorted_indices_by_ant_solution = np.argsort(distances)
        best_ant = sorted_indices_by_ant_solution[0]

        # Perform local pheromone update on the ant's path
        for ant in range(ants_number):
            if distances[ant] != np.inf:
                for i in range(len(routes[ant])-1):
                    current_path_node = routes[ant][i]
                    next_path_node = routes[ant][i+1]
                    indx_next_node = graph_map["connections"][current_path_node].index(next_path_node)
                    pheromone_graph[current_path_node][indx_next_node] = ((1 - global_evap_rate) * pheromone_graph[current_path_node][indx_next_node]) + (global_evap_rate * (1 / distances[ant]))

                    if(best_ant == ant): # Deposit pheromone on the paths of the best ant
                        pheromone_graph[current_path_node][indx_next_node] = ((1 - global_evap_rate) * pheromone_graph[current_path_node][indx_next_node]) + (global_evap_rate * (1 / distances[best_ant]))

        # Calculate maximum pheromone limit
        if ant_paths[sorted_indices[0]][-1] != float('inf'):
            f_max = p * ant_paths[sorted_indices[0]][-1]

        # Deposit pheromone on the paths of the 3 best ants
        for best in range(3):
            if ant_paths[sorted_indices[best]][-1] != float('inf'):
                for i in range(len(ant_paths[sorted_indices[best]]) - 2):
                    pheromone_matrix[ant_paths[sorted_indices[best]][i+1], ant_paths[sorted_indices[best]][i]] += p * (1 / ant_paths[sorted_indices[best]][-1])

        # Maintain minimum and maximum pheromone levels across the graph
        pheromone_matrix[pheromone_matrix < initial_pheromone] = initial_pheromone
        pheromone_matrix[pheromone_matrix > f_max] = f_max

        # Check stopping criterion of the algorithm
        ant_distances = ant_distances[ant_distances != float('inf')]  # Paths lost by ants do not count
        if len(ant_distances) > 0:
            _, number_ants_following_path = np.mode(ant_distances)  # Most followed path by ants (and how many ants follow it)

        # Correct stagnation conditions
        if epochs == max_epochs:
            pheromone_matrix = initial_pheromone + np.zeros((n, m))
            max_epochs *= 2

        epochs += 1

    # Return the optimal path
    path = ant_paths[0][:-1]  # Exclude the last element which is the total distance
    costo_MejorGlobal = ant_paths[0][-1]  # Total distance of the optimal path
    t = time.time() - tic
    return path, costo_MejorGlobal, t, epochs


In [30]:
start_node = 4
end_node = 52
num_ants = settings["ants"]
evaporation_rate =0.8
initial_pheromone_lvl =settings["f_ini"]
heuristic_weight=settings["alfa"]
pheromone_weight=settings["beta"]

epomax = settings["epomax"]
local_evap_rate=0.5
transition_prob=settings["transition_probability"]

solution_path,cost,time,epochs = ABW(map_graph, start_node, end_node, num_ants, evaporation_rate, epomax, initial_pheromone_lvl, heuristic_weight, pheromone_weight)

print(solution_path)
print(cost)
print(time)
print(epochs)

[4, 14, 24, 34, 44, 43, 53, 52]
0.7
8.022258520126343
142
